In [4]:
import networkx as nx
import random 
random.seed(0)
import numpy as np
np.random.seed(0)

In [5]:
G=nx.erdos_renyi_graph(10,0.3, seed=1, directed=False)

In [12]:
def next_node (previous,current, p,q):
    neighbours=list(G.neighbors(current))
    alphas=[]
    for neighbor in neighbours:
        if neighbor==previous:
            alpha=1/p
        elif G.has_edge(neighbor,previous):
            alpha=1
        else:
            alpha=1/q
        alphas.append(alpha)
    probs=[alpha/sum(alphas) for alpha in alphas]
    next_nodes=np.random.choice(neighbours, size=1, p=probs)[0]
    return next_nodes
            

In [13]:
def random_walk(start, length, p,q):
    walk=[start]
    for i in range(length):
        current=walk[-1]
        previous=walk[-2] if len(walk)>1 else None
        next_n=next_node(previous,current,p,q)
        walk.append(next_n)
    return [str(x) for x in walk]

In [14]:
random_walk(0,8,1,1)

['0', '4', '7', '6', '4', '5', '4', '5', '6']

In [16]:
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score

C:\Users\hrida\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
G=nx.karate_club_graph()

In [18]:
labels=[]
for node in G.nodes:
    label=G.nodes[node]['club']
    labels.append(1 if label=='Officer' else 0)

In [19]:
walks=[]
for node in G.nodes:
    for _ in range(80):
        walks.append(random_walk(node,10,3,2))

In [20]:
node2vec=Word2Vec(walks, hs=1, sg=1, vector_size=100, window=10, workers=2, min_count=1, seed=0)

In [21]:
node2vec.train(walks, total_examples=node2vec.corpus_count, epochs=30)

(186344, 897600)

In [22]:
train_mask=[2,4,6,8,10,12,14,16,18,20,22,24]
train_mask_str=[str(x) for x in train_mask]
test_mask=[0,1,3,5,7,9,11,13,15,17,19,21,23,25,26,27,28,29,30,31,32,33]
test_mask_str=[str(x) for x in test_mask]
labels=np.array(labels)

In [23]:
clf=RandomForestClassifier(random_state=0)
clf.fit(node2vec.wv[train_mask_str], labels[train_mask])

RandomForestClassifier(random_state=0)

In [ ]:
y_pred=clf.predict(node2vec.wv[test_mask_str])
acc=accuracy_score(y_pred, labels[test_mask])